In [2]:
import tensorflow as tf
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

tf.python.control_flow_ops = tf


Using TensorFlow backend.


In [3]:
from src.data import Data
from src.plothelp import *

In [4]:
data = Data("img")
data.load_pickle("28x28.pickle")
X_train, X_test, y_train, y_test = data.train_test_split()

In [5]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
X_train = X_train.reshape(X_train.shape[0],1,28,28)
X_test = X_test.reshape(X_test.shape[0],1,28,28)

print(X_train.shape)
print(X_test.shape)

(4698, 1, 28, 28)
(522, 1, 28, 28)


In [6]:
def oneHot(y_train, y_test):
    from sklearn import preprocessing
    
    le = preprocessing.LabelEncoder()
    le.fit(y_train)
    numeric_labels =le.transform(y_train)
    numeric_test_labels = le.transform(y_test)
    n_classes =4
    y_one_hot = np.eye(4)[numeric_labels]
    y_test_code = np.eye(4)[numeric_test_labels]
    
    return y_one_hot, y_test_code
    
    

In [7]:
y_oh, y_test_oh= oneHot(y_train, y_test)

In [8]:
# Create the model
model = Sequential()
model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(1,28,28)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(3,64,64)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
# Compile model
lrate = 0.01
nb_epoch=20
decay = lrate/nb_epoch
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 26, 26)    640         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 13, 13)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 11, 11)    36928       maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 64, 5, 5)      0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [ ]:
# Fit the model
model.fit(X_train, y_oh, validation_data=(X_test, y_test_oh), nb_epoch=20, batch_size=100, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test_oh, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 4698 samples, validate on 522 samples
Epoch 1/20
4698/4698 [==============================] - 94s - loss: 12.1375 - acc: 0.2444 - val_loss: 11.8261 - val_acc: 0.2663